Тренировка модели

In [1]:
from tensorflow.compat.v1 import keras
import tensorflow as tf

config = tf.compat.v1.ConfigProto( device_count = {'GPU': 1 , 'CPU': 8} ) 
sess = tf.compat.v1.Session(config=config) 
tf.compat.v1.keras.backend.set_session(sess)

In [2]:
import requests

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)


file_id = '1gN_NONnkggeIxvxplQ2WFuqAFw6Uvl95'
destination = 'AIJ_2GIS_ru.zip'
download_file_from_google_drive(file_id, destination)

In [3]:
import zipfile
with zipfile.ZipFile('/content/AIJ_2GIS_ru.zip', 'r') as zip_ref:
  zip_ref.extractall('/content/data')

In [4]:
import zipfile
with zipfile.ZipFile('/content/data/AIJ_2GIS_ru/AIJ_2gis.zip', 'r') as zip_ref:
  zip_ref.extractall('/content/photos')

In [5]:
import numpy as np
from PIL import Image, ImageFilter
import cv2
from google.colab.patches import cv2_imshow
import pandas as pd
from tensorflow import keras

In [6]:
train_df = pd.read_csv('/content/photos/AIJ_2gis/train.csv')
train_df['cat_label'] = ''
train_df.cat_label = train_df.label.astype('category').cat.codes
train_df

,filename,label,cat_label
0,train_images/c8e6dec4-79f0-469f-be8b-9881f5943...,3_24+10,52
1,train_images/75fac931-0467-404a-8195-23d1a8875...,3_24+10,52
2,train_images/ee7074aa-a71a-4fbf-b7ca-4dc225214...,3_24+10,52
3,train_images/f0e6b784-de28-4fd6-a1ff-97b1fa96e...,3_24+10,52
4,train_images/246cec0f-a900-45d1-b097-c8d2c3a4c...,3_24+10,52
...,...,...,...
46058,train_images/612f1f0e-2f2c-4143-b4bc-3e6cd57fc...,5_15_2+5,120
46059,train_images/186a429c-2c0f-4594-96fc-695e85569...,5_15_2+5,120
46060,train_images/ae246a42-590c-4e5b-b6d9-bdeee17fd...,5_15_2+5,120
46061,train_images/f405cbf4-e06d-4e0a-99f5-6a58308ce...,5_15_2+5,120


In [7]:
y_train = train_df['cat_label']
y_train = np.array(y_train)
y_train_cat_pre = keras.utils.to_categorical(y_train, 182)
y_train_cat_pre[500]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [8]:
len(train_df)

46063

In [9]:
images = []
y_train_cat = []
y_train_orig = []

for i in range(len(train_df)):
  try:
    images.append(Image.open('/content/photos/AIJ_2gis/' + train_df['filename'].iloc[i]).resize((40, 40), Image.BILINEAR))
    y_train_cat.append(y_train_cat_pre[i])
    y_train_orig.append(train_df['label'].iloc[i])
  except:
    pass

In [10]:
x_train = images.copy()
for i in range(len(x_train)):
  x_train[i] = np.asarray(x_train[i])
x_train

[array([[[113, 108, 104],
         [113, 106, 100],
         [115, 106, 101],
         ...,
         [119, 106,  98],
         [127, 110, 102],
         [142, 122, 113]],
 
        [[102,  97,  93],
         [ 92,  85,  79],
         [123, 114, 109],
         ...,
         [111,  99,  90],
         [112,  98,  89],
         [129, 109, 102]],
 
        [[100,  95,  89],
         [ 92,  84,  79],
         [145, 135, 131],
         ...,
         [107,  97,  88],
         [107,  94,  86],
         [125, 107, 100]],
 
        ...,
 
        [[182, 148,  75],
         [161, 131,  73],
         [144, 120,  77],
         ...,
         [ 90,  99, 104],
         [115, 124, 129],
         [144, 154, 158]],
 
        [[164, 132,  55],
         [168, 141,  84],
         [152, 129,  92],
         ...,
         [ 89,  99, 106],
         [115, 126, 132],
         [141, 152, 158]],
 
        [[171, 142,  72],
         [160, 135,  87],
         [124, 104,  75],
         ...,
         [ 87,  97, 106],
  

In [11]:
x_train = np.array(x_train)
x_train = x_train.astype('float16')

In [12]:
x_train = (x_train - 127.5) / 127.5

In [13]:
x_train.shape

(46063, 40, 40, 3)

In [14]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout

model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=(40,40,3)))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(182, activation='softmax'))

model.compile(
    loss='categorical_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy']
)

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 36, 36, 32)        2432      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 17, 17, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 17, 17, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 15, 15, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 7, 64)          0

In [16]:
y_train_cat = np.array(y_train_cat)
y_train_cat

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [17]:
Cells=np.array(x_train)
labels=np.array(y_train_cat)

s=np.arange(Cells.shape[0])
np.random.seed(43)
np.random.shuffle(s)
Cells=Cells[s]
labels=labels[s]

In [20]:
(X_train,X_val)=Cells[(int)(00*len(labels)):],Cells[:(int)(00*len(labels))]
(y_train,y_val)=labels[(int)(00*len(labels)):],labels[:(int)(00*len(labels))]

In [23]:
X_train.shape

(46063, 40, 40, 3)

In [25]:
his = model.fit(X_train, y_train, batch_size=32, epochs=10)

Epoch 1/10
1440/1440 [==============================] - 11s 7ms/step - loss: 0.0891 - accuracy: 0.9764
Epoch 2/10
1440/1440 [==============================] - 10s 7ms/step - loss: 0.0885 - accuracy: 0.9764
Epoch 3/10
1440/1440 [==============================] - 11s 7ms/step - loss: 0.0784 - accuracy: 0.9789
Epoch 4/10
1440/1440 [==============================] - 10s 7ms/step - loss: 0.0795 - accuracy: 0.9797
Epoch 5/10
1440/1440 [==============================] - 10s 7ms/step - loss: 0.0842 - accuracy: 0.9786
Epoch 6/10
1440/1440 [==============================] - 11s 7ms/step - loss: 0.0819 - accuracy: 0.9789
Epoch 7/10
1440/1440 [==============================] - 10s 7ms/step - loss: 0.0790 - accuracy: 0.9794
Epoch 8/10
1440/1440 [==============================] - 10s 7ms/step - loss: 0.0801 - accuracy: 0.9787
Epoch 9/10
1440/1440 [==============================] - 11s 7ms/step - loss: 0.0864 - accuracy: 0.9791
Epoch 10/10
1440/1440 [==============================] - 10s 7ms/step - l

In [ ]:
model.save('TraficSignRecognition.h5')

Создание первого словаря

In [43]:
a = train_df['cat_label'].unique()
b = train_df['label'].unique()

dct = {}

for i in range(len(a)):
  dct[str(a[i])] = str(b[i])

In [44]:
dct['126']

'5_19'

In [94]:
dct

{'0': '1_1',
 '1': '1_11_1',
 '10': '1_18',
 '100': '4_5_5',
 '101': '4_5_6',
 '102': '4_8_1',
 '103': '4_8_2',
 '104': '4_8_3',
 '105': '4_8_4',
 '106': '4_8_5',
 '107': '4_8_6',
 '108': '5_1',
 '109': '5_11_1',
 '11': '1_19',
 '110': '5_12_1',
 '111': '5_13_1',
 '112': '5_13_2',
 '113': '5_14',
 '114': '5_14_1',
 '115': '5_14_2',
 '116': '5_14_3',
 '117': '5_15_2+1',
 '118': '5_15_2+2',
 '119': '5_15_2+3',
 '12': '1_2',
 '120': '5_15_2+5',
 '121': '5_15_3',
 '122': '5_15_5',
 '123': '5_16',
 '124': '5_17',
 '125': '5_18',
 '126': '5_19',
 '127': '5_2',
 '128': '5_20',
 '129': '5_21',
 '13': '1_20_1',
 '130': '5_22',
 '131': '5_23_2',
 '132': '5_24_2',
 '133': '5_29',
 '134': '5_3',
 '135': '5_31+10',
 '136': '5_31+20',
 '137': '5_31+30',
 '138': '5_31+40',
 '139': '5_31+5',
 '14': '1_20_2',
 '140': '5_31+50',
 '141': '5_32',
 '142': '5_33',
 '143': '5_4',
 '144': '5_5',
 '145': '5_6',
 '146': '5_7_1',
 '147': '5_7_2',
 '148': '5_8',
 '149': '5_9',
 '15': '1_20_3',
 '150': '6_15_1',
 

Создание второго словаря

In [53]:
from bs4 import BeautifulSoup

In [67]:
with open("site.html", "r") as f:
    
  contents = f.read()
 
  soup = BeautifulSoup(contents, 'lxml')
  names =  soup.findAll(attrs={'class':'b-title b-title_type_h4'})

In [81]:
len(names)

250

In [92]:
res = {}
for i in names:
  a = str(i)  
  a = a[a.index('Знак'):]
  a = a[a.index(' ')+1:]
  b = a[:a.index(' ')-1].replace('.','_')
  a = a[a.index(' ')+1:a.index('<')]

  if b[len(b)-1] == '_':
    b = b[:len(b)-1]

  res[b]=a

In [93]:
res

{'1_1': 'Железнодорожный переезд со шлагбаумом',
 '1_10': 'Выезд на набережную',
 '1_11_1': 'Опасный поворот (правый)',
 '1_11_2': 'Опасный поворот (левый)',
 '1_12_1': 'Опасные повороты (с первым поворотом направо)',
 '1_12_2': 'Опасные повороты (с первым поворотом налево)',
 '1_13': 'Крутой спуск',
 '1_14': 'Крутой подъем',
 '1_15': 'Скользкая дорога',
 '1_16': 'Неровная дорога',
 '1_17': ' Искусственная неровность',
 '1_18': 'Выброс гравия',
 '1_19': 'Опасная обочина',
 '1_2': 'Железнодорожный переезд без шлагбаума',
 '1_20_1': ' Сужение дороги. ',
 '1_20_2': ' Сужение дороги ',
 '1_20_3': 'Сужение дороги',
 '1_21': 'Двустороннее движение',
 '1_22': 'Пешеходный переход',
 '1_23': ' Дети',
 '1_24': 'Пересечение с велосипедной дорожкой или велопешеходной дорожкой',
 '1_25': 'Дорожные работы',
 '1_26': 'Перегон скота',
 '1_27': 'Дикие животные',
 '1_28': 'Падение камней',
 '1_29': 'Боковой ветер',
 '1_30': 'Низколетящие самолеты',
 '1_31': 'Тоннель',
 '1_32': 'Затор',
 '1_33': 'Прочие 